##Decision trees

In [7]:
from typing import NamedTuple,Optional

In [41]:
class Candidate(NamedTuple):
    level:str
    lang:str
    tweets:bool
    phd:bool
    did_well:Optional[bool]=None

In [40]:
dataset=[Candidate('Senior','Java',False,False,False),
         Candidate('Senior','Java',False,True,False),
         Candidate('Mid','Python',False,False,True),
         Candidate('Junior','Python',False,False,True),
         Candidate('Junior','R',True,False,True),
         Candidate('Junior','R',True,True,False),
         Candidate('Mid','R',True,True,True),
         Candidate('Senior','Python',False,False,False),
         Candidate('Senior','R',True,False,True),
         Candidate('Junior','Python',True,False,True),
         Candidate('Senior','Python',True,True,True),
         Candidate('Mid','Python',False,True,True),
         Candidate('Mid','Java',True,False,True),
         Candidate('Junior','Python',False,True,False),
]

In [25]:
from typing import Dict,TypeVar,Any,List
from  collections import  defaultdict,Counter
import math

In [46]:
T=TypeVar("T")

def entropy(class_probabilities:List[float])->float:
    return sum(-p*math.log(p,2)
               for p in class_probabilities
               if p>0)

def class_probabilities(labels:List[Any])-> List[float]:
    total_count=len(labels)
    return [count/total_count
            for count in Counter(labels).values()]

def data_entropy(labels:List[Any])->float:
    return entropy(class_probabilities(labels))

def partition_entropy(subsets:List[List[Any]])->float:
    total_count=sum(len(subset) for subset in subsets)

    return  sum(data_entropy(subset)*len(subsets)/total_count
                for subset in subsets)

def partition_by(inputs:List[T],attribute:str)->Dict[Any,List[T]]:
    partitions: Dict[Any,List[T]]=defaultdict(list)
    for input in inputs:
        key=getattr(input,attribute)
        partitions[key].append(input)
    return partitions

def partition_entropy_by(inputs:List[Any],attribute:str,label_attribute:str)->float:
    partitions=partition_by(inputs,attribute)
    labels=[[getattr(input,label_attribute) for input in partition]
            for partition in partitions.values()]

    return partition_entropy(labels)


In [34]:
data_entropy([3,4,4,4])

dict_values([1, 3])


0.8112781244591328

In [47]:
for key in ['level','lang','tweets','phd']:
    print(key,partition_entropy_by(dataset,key,'did_well'))

level 0.4161216833377151
lang 0.5555773986600543
tweets 0.22527155923093986
phd 0.2587540177798761


In [43]:
senior_inputs=[input for input in dataset if input.level =="Senior"]

In [44]:
from typing import NamedTuple,Union,Any

class Leaf(NamedTuple):
    value:Any

class Split(NamedTuple):
    attribut:str
    subtrees:dict
    default_value:Any=None

DecisionTree=Union[Leaf,Split]

In [57]:
hiring_tree=Split('level',{
    'Junior':Split('phd',{
        False:Leaf(True),
        True:Leaf(False)
    }),
    'Mid':Leaf(True),
    'Senior':Split('tweets',{
        False:Leaf(False),
        True:Leaf(True)
    })

})

def classify(tree:DecisionTree,input:Any)->Any:
    if isinstance(tree,Leaf):
        return tree.value

    subtree_key=getattr(input,tree.attribut)

    if subtree_key not in tree.subtrees:
        return tree.default_value

    subtree=tree.subtrees[subtree_key]

    return classify(subtree,input)


def build_tree_id3(inputs:List[Any],
                   split_attributes:List[str],
                   target_attribute:str)-> DecisionTree:
    label_counts=Counter(getattr(input,target_attribute)
                         for input in inputs)

    most_common_label=label_counts.most_common(1)[0][0]

    if len(label_counts)==1:
        return Leaf(most_common_label)

    if not split_attributes:
        return Leaf(most_common_label)

    def split_entropy(attribute:str)->float:
        return partition_entropy_by(inputs,attribute,target_attribute)

    best_attribute=min(split_attributes,key=split_entropy)

    partitions=partition_by(inputs,best_attribute)

    new_attribute=[a for a in split_attributes if a!= best_attribute]

    subtrees={attribute_value:build_tree_id3(subset,
                                             new_attribute,
                                             target_attribute)
              for attribute_value,subset in partitions.items()

    }

    return  Split(best_attribute,subtrees,
                  default_value=most_common_label)

In [54]:
tree=build_tree_id3(dataset,['level','lang','tweets','phd'],'did_well')

In [56]:
classify(tree,Candidate('Junior','R',True,True))

False